<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Model_based_Pacman_ATARI_labels_020420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

In [1]:
!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/mila-iqia/atari-representation-learning.git to /tmp/pip-req-build-lgscm3ua
  Running command git clone -q git://github.com/mila-iqia/atari-representation-learning.git /tmp/pip-req-build-lgscm3ua
  Created wheel for atariari: filename=atariari-0.0.1-cp36-none-any.whl size=46584 sha256=9a3da5866687fe6509571906cbe7a2019fd5a416ae7a0503bddfc788b8bf8c86
  Stored in directory: /tmp/pip-ephem-wheel-cache-5am0rzfv/wheels/3d/69/51/5e436e5ae566c5b4dec5c53e65396d516459877a42a11d7aa4
Successfully built atariari
  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-skgadwlx
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-skgadwlx
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=009ec331b8fe4dfb313ef41a87901be851cbd2974aa0b963c747a319ae92bb52
  Stored in directory: /tmp/pip-ephem-wheel-cache-iu5fmnwx/wheels/91/52/02/ec5c530fd76d56a66

In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time

from atariari.benchmark.envs import *
from torch.autograd import Variable

import gym
from atariari.benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [10]:
!wandb login #############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [11]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")

W&B Run: https://app.wandb.ai/nerdk312/ATARI_LABELS_MPC/runs/5t7drc4e

# Model-based RL

In [6]:
env = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4'))
obs = env.reset()
original_info = env.labels() # Nawid - This only contains the labels
print(original_info)
i = 0
t = 0
while True:
    _,_,done,info = env.step(env.action_space.sample())
    info_labels = info['labels']
    t += 1
    if not original_info['num_lives'] == info_labels['num_lives']:
        print('change in num lives')
        original_info = info_labels
        i = 0
        print('t value',t)

    #print('info',info['labels']) # Nawid - Need to specify the labels
    if original_info == info_labels:
        i += 1
        print(i)

    if done:
        print('overall time',t)
        break

'''
next_ale_lives = np.expand_dims(np.array([i['ale.lives'] for i in infos]),1) # Nawid - Extracts the information related to the number of lives
dones =  ale_lives - next_ale_lives # Nawid -  Finds the change in lives after a timestep
dones[dones< 1] = 0 # Nawid - Makes it so that the situation where it resets does not have a change in 3 for the values
ale_lives = next_ale_lives # Nawid- Update the ale lives
'''

{'enemy_sue_x': 88, 'enemy_inky_x': 88, 'enemy_pinky_x': 88, 'enemy_blinky_x': 88, 'enemy_sue_y': 80, 'enemy_inky_y': 80, 'enemy_pinky_y': 80, 'enemy_blinky_y': 50, 'player_x': 88, 'player_y': 98, 'fruit_x': 0, 'fruit_y': 0, 'ghosts_count': 3, 'player_direction': 3, 'dots_eaten_count': 0, 'player_score': 0, 'num_lives': 2}
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196

"\nnext_ale_lives = np.expand_dims(np.array([i['ale.lives'] for i in infos]),1) # Nawid - Extracts the information related to the number of lives\ndones =  ale_lives - next_ale_lives # Nawid -  Finds the change in lives after a timestep\ndones[dones< 1] = 0 # Nawid - Makes it so that the situation where it resets does not have a change in 3 for the values\nale_lives = next_ale_lives # Nawid- Update the ale lives\n"

In [7]:
env = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4'))

#print(original_info)
i = 0
for k in range(2):
    obs = env.reset()
    original_info = env.labels() # Nawid - This only contains the labels
    while True:
        _,_,done,info = env.step(env.action_space.sample())
        info_labels = info['labels']
    
        if not original_info['num_lives'] == info_labels['num_lives']:
            
            print('change in num lives')
            print('info_labels',info_labels)
            original_info = info_labels
            i = 0

        #print('info',info['labels']) # Nawid - Need to specify the labels
        if original_info == info_labels:
            i += 1
            print(i)

        if done:
            print('episode is done')
            i = 0
            break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
change in num lives
info_labels {'enemy_sue_x': 88, 

# Hyperparameters

In [0]:
ENV_NAME = 'MsPacmanNoFrameskip-v4'

#feature_size = all_defaults['feature_size'] # Nawid- Dimensionality of the representation
# workers = 8 # Nawid - Choosing the number of workers for the network

# Main loop hyperp
AGGR_ITER = 10
STEPS_PER_AGGR = 500

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 2000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Supervised Model Hyperp
ENV_LEARNING_RATE = 1e-4
REW_LEARNING_RATE = 1e-4
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  55

# Controller Hyperp
HORIZON_LENGTH = 5
NUM_ACTIONS_SEQUENCES = 200

observation_channels = 1
action_dim = 1
n_actions = 5 #  9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}.{}.{}".format(now.day, now.hour, now.minute, now.second)

# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

# Model setup

In [0]:
class EarlyStopping_loss(object):
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=5, verbose=False, wandb=None, name=""):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = 1e11 # Nawid - Set a very high initial best loss
        self.name = name
        self.wandb = wandb

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score >= self.best_score: # Nawid - Inverse signs to take into minimising loss instead of maximising accuracy
            self.counter += 1
            print(f'EarlyStopping for {self.name} counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                print(f'{self.name} has stopped')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(
                f'Validation loss decreased/improved for {self.name}  ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')

        save_dir = self.wandb.run.dir
        torch.save(model.state_dict(), save_dir + "/" + self.name + ".pt")
        self.wandb.save(save_dir + "/" + self.name + ".pt")
        self.val_loss_min = val_loss


In [0]:
def one_hot(i):
    a = np.zeros(n_actions, 'uint8')
    a[i] = 1
    return a

In [0]:
class NNDynamicsModel(nn.Module):
    '''
    Model that predict the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicsModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, obs_output_dim)
        )
    def forward(self, x):
        return self.mlp(x.float())

class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, reward_output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x.float())

def model_MSEloss(y_pred,y_truth, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

def model_CEloss(y_pred,y_truth,device):
    '''
    Compute the CEloss
    '''
    y_truth = torch.Tensor(np.array(y_truth)).to(device)
    return F.cross_entropy(y_pred, y_truth)

def model_BCEloss(y_pred,y_truth,device):
    '''
    Compute the BCE (Binary cross entropy)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.binary_cross_entropy(y_pred.view(-1).float(), y_truth.view(-1))

# Data collection and preprocessing

In [0]:
class Data_collection():
    def __init__(self,ENV_NAME,state_mode = 'ATARIARI', encoder = None):
        self.ENV_NAME = ENV_NAME
        self.state_mode = state_mode
        self.encoder = encoder
        self.repeated_initial = True # Nawid - set repeated initial as true initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def gather_random_trajectories(self,num_traj):

        dataset_random = []
        #Env name could either be the RAM case or the generic case
        env = AtariARIWrapper(gym.make(self.ENV_NAME)) 
        #i = 0
        for n in range(num_traj):
            # Initial set up
            obs = env.reset()
            self.repeated_initial = True # Nawid- Used to represent the initial state
            initial_info_labels = env.labels()
            info_labels = env.labels() # Nawid -  Used to get the current state
            #print('trajectory number',n)
            
            while True:
                # Choosing action and env step
                sampled_action = np.random.randint(0,n_actions)
                sampled_action_one_hot = one_hot(sampled_action)
                next_obs, reward, done, next_info = env.step(sampled_action)
                next_info_labels = next_info['labels']
                # self.repeated initial is set to true at first and it is is turned to true in the reward_collection class when done is true

                if self.repeated_initial: # If the initial state is repeating
                    if initial_info_labels == next_info_labels: # Current state is still the same as the initial state
                        pass
                    else:
                        # New state achieved, so save data
                        self.repeated_initial = False
                    
                        state, next_state = self.state_collection(obs, next_obs,info_labels, next_info_labels)
                        reward, pacman_done = self.reward_collection(reward, done, info_labels, next_info_labels)
                        dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                    
                        obs = next_obs
                        info_labels = next_info_labels

                else:
                    # Save data as this is when the initial state and the next state are not identical
                    state, next_state = self.state_collection(obs, next_obs,info_labels, next_info_labels)
                    reward, pacman_done = self.reward_collection(reward, done, info_labels, next_info_labels) # Nawid - Changed name to pacman done as the variable done is used to exit the loop
                    dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                
                    obs = next_obs
                    info_labels = next_info_labels
                    ''' Used to test whether the done behaviour was correct
                    if pacman_done:
                        i +=1 
                        print('pacman done', i)
                    '''

                if done:
                    break 
        '''               
        #dataset_random = np.array(dataset_random)
        first_dataset = np.array(dataset_random[0])
        print(first_dataset[0])
        print('zero',dataset_random[0])
        print('One',dataset_random[1])
        print('Two',dataset_random[2])
        print('Three',dataset_random[3])
        print('Four',dataset_random[4])
        #print(dataset_random.shape)
        '''
        return dataset_random
    

    def state_collection(self,obs, next_obs, info_labels, next_info_labels):
        if self.state_mode == 'ATARIARI':
            state = []
            next_state = []
            i = 0 
            for key in info_labels :
                if i < 14: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    next_state.append(next_info_labels[key])
                    i +=1
                else:
                    state = np.array(state)
                    next_state = np.array(next_state)
                    return state, next_state
    
        elif self.state_mode == 'STDIM': # Encoding is the state
            assert self.encoder is not None
            with torch.no_grad():
                self.encoder.eval()
                state = self.encoder(obs.float().to(self.device) / 255)
                next_state = self.encoder(next_obs.float().to(self.device) / 255)
                return state.cpu().detach().numpy(), next_state.cpu().detach().numpy()

        else: # Image observations are the state or the RAM labels
            return obs, next_obs

    def reward_collection(self,reward,done,info_labels,next_info_labels):
        # checks whether there has been a change in lives- change done to 1, otherwise the done should be fine regardless
        if not info_labels['num_lives'] == next_info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
            #print('change in lives')
            done = 1 
            self.repeated_initial = True
        return reward, done

    def unison_shuffled_copies(self,*args): # Nawid- Randomises all the different values, using *args to use a variable number of parameters so i can shuffle many different values at once
        p = np.random.permutation(len(args[0]))
        shuffled = [i[p] for i in args]
        return shuffled
    
    def collate_data(self,random_dataset, rl_dataset):
        if len(rl_dataset) > 0:
                
            random_dataset = np.array(random_dataset)
            rl_dataset = np.array(rl_dataset)
            print('random_dataset', random_dataset.shape)
            print('rl dataset', rl_dataset.shape)
        
            d_concat = np.concatenate([random_dataset, rl_dataset], axis=0)
            print('d_concat',d_concat.shape)        
        else:
            d_concat = np.array(random_dataset)
        
        num_examples_added = len(d_concat)

        # Split the dataset into train(80%) and test(20%)
        D_train = d_concat[:int(-num_examples_added*1/5)]
        D_valid = d_concat[int(-num_examples_added*1/5):]

        print("len(D):", len(d_concat), 'len(Dtrain)', len(D_train))

        # Shuffle the dataset
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]

        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_act = np.array([act for _,_,_,_,act in D_train])

        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        y_env_train = y_env_train - np.array([obs for obs,_,_,_,_ in D_train]) # y(state) = s(t+1) - s(t)

        # Reward's output
        y_rew_train = np.array([[D] for _,_,_,D,_ in D_train])
    
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val - np.array([obs for obs,_,_,_,_ in D_valid]) # y(state) = s(t+1) - s(t)

        # Reward output
        y_rew_val = np.array([[D] for _,_,_,D,_ in D_valid])
    
        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        rew_train_data, rew_val_data = (X_train_obs, X_train_act, y_rew_train), (X_val_obs, X_val_act, y_rew_val)

        return env_train_data, env_val_data, rew_train_data, rew_val_data 
    



data_collector = Data_collection('MsPacmanNoFrameskip-v4')
random_data = data_collector.gather_random_trajectories(5)

In [0]:
def state_collection(obs, next_obs, info_labels, next_info_labels,mode='ATARIARI', encoder = None):
    if mode == 'ATARIARI':
        state = []
        next_state = []
        i = 0 
        for key in info_labels :
            if i < 14: # Nawid - Only the first 14 info are crucial I believe
                state.append(info_labels[key])
                next_state.append(info_labels[key])
                i +=1
            else:
                state = np.array(state)
                next_state = np.array(next_state)
                return state, next_state
    
    elif mode == 'STDIM': # Encoding is the state
        assert encoder is not None
        with torch.no_grad():
            encoder.eval()
            states = encoder(obs.float().to(device) / 255)
            next_states = encoder(next_obs.float().to(device) / 255)
            return states.cpu().detach().numpy(), next_states.cpu().detach().numpy()

    states.cpu().detach().numpy()

    else: # Image observations are the state or the RAM labels
        return obs, next_obs

'''
env = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4')) 
info_labels = env.labels()
#print(info_labels)
state = []
print(info_labels)
for key in info_labels:
    state.append(info_labels[key])

state = np.array(state)
compact_state = state[0:14]
print(compact_state)
'''

"\nenv = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4')) \ninfo_labels = env.labels()\n#print(info_labels)\nstate = []\nprint(info_labels)\nfor key in info_labels:\n    state.append(info_labels[key])\n\nstate = np.array(state)\ncompact_state = state[0:14]\nprint(compact_state)\n"

In [0]:
def gather_random_trajectories(num_traj,ENV_NAME,mode = 'ATARIARI', encoder = None):
    '''
    states_list = []
    next_states_list = []
    rewards_list = []
    dones_list = []
    actions_list = []
    '''
    dataset_random = []
    #Env name could either be the RAM case or the generic case
    env = AtariARIWrapper(gym.make(ENV_NAME)) 

    for n in range(num_traj):
        obs = env.reset()
        repeated_initial = True # Nawid- Used to represent the initial state
        initial_info_labels = env.labels()
        info_labels = env.labels() # Nawid -  Used to get the current state
        while True:
            sampled_action = np.random.randint(0,n_actions)
            sampled_action_one_hot = one_hot(sampled_action)
            next_obs, reward, done, next_info = env.step(sampled_action)
            next_info_labels = next_info['labels']

            if repeated_initial: # If the initial state is repeating
                if initial_info_labels == next_info_labels: # Current state is still the same as the initial state
                    pass
                else:
                    repeated_initial = False
                    
                    state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                    reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                    dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                    
                    obs = next_obs
                    info_labels = next_info_labels

                    # Go into a new state and from the previous state and the current state as this is a transition between the states

            else:
                # Save data as this is when the initial state and the next state are not identical
                state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                
                obs = next_obs
                info_labels = next_info_labels

                if not initial_info_labels['num_lives'] == info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
                    print('change in lives')
                    repeated_initial = True



In [0]:
def gather_random_trajectories(steps,ENV_NAME,encoder):
    
    states_list = []
    next_states_list = []
    rewards_list = []
    dones_list = []
    actions_list = []
    #dataset_random = []
    
    envs = make_vec_envs(ENV_NAME,1,8,downsample=False)
    obs = envs.reset()
    ale_lives = np.full((workers,1),3) 
    for i in range(steps//8):
        #actions = torch.tensor(np.array([np.random.randint(1, envs.action_space.n) for _ in range(workers)])).unsqueeze(1)
        actions = torch.tensor(np.array([np.random.randint(1, n_actions) for _ in range(workers)])).unsqueeze(1) # Nawid - Choosing actions on a smaller action space
        
        next_obs, rewards, _, infos = envs.step(actions)

        next_ale_lives = np.expand_dims(np.array([i['ale.lives'] for i in infos]),1) # Nawid - Extracts the information related to the number of lives
        dones =  ale_lives - next_ale_lives # Nawid -  Finds the change in lives after a timestep
        dones[dones< 1] = 0 # Nawid - Makes it so that the situation where it resets does not have a change in 3 for the values
        ale_lives = next_ale_lives # Nawid- Update the ale lives
        
        with torch.no_grad():
            encoder.eval()
            states = encoder(obs.float().to(device) / 255)
            next_states = encoder(next_obs.float().to(device) / 255)

        sampled_action_one_hot = np.array([one_hot(action) for action in actions]) # Nawid - Looks at each action in the vectorised environment and obtains the different
        
        states_list.append(states.cpu().detach().numpy())
        next_states_list.append(next_states.cpu().detach().numpy())
        rewards_list.append(rewards.cpu().numpy())
        dones_list.append(dones)
        # dones_list.append(np.expand_dims(dones,1))
        actions_list.append(sampled_action_one_hot)
        
        #dataset_random.append([states.cpu().detach().numpy(), next_states.cpu().detach().numpy(), rewards.cpu().numpy(), np.expand_dims(dones,1), sampled_action_one_hot])
    envs.close()
    encoder.train()
    
    states_list = np.concatenate(states_list)
    next_states_list = np.concatenate(next_states_list)
    rewards_list = np.concatenate(rewards_list)
    dones_list = np.concatenate(dones_list)
    actions_list =  np.concatenate(actions_list)
    return states_list, next_states_list, rewards_list, dones_list, actions_list
    

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = ImpalaCNN(1,all_defaults)
encoder.to(device)
states_list,next_states_list, rewards_list, dones_list, actions_list = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
#random_dataset = gather_random_trajectories(1000,'MsPacmanNoFrameskip-v4',encoder)


rewards list (10000, 1)
dones list (10000, 1)


In [0]:
def unison_shuffled_copies(*args): # Nawid- Randomises all the different values, using *args to use a variable number of parameters so i can shuffle many different values at once
    p = np.random.permutation(len(args[0]))
    shuffled = [i[p] for i in args]
    return shuffled

In [0]:
def collate_data(random_dataset, rl_dataset, num_examples_added):
    if len(rl_dataset)>0:
        rand_states, rand_next_states, rand_rewards, rand_dones, rand_actions = random_dataset
        rl_states, rl_next_states, rl_rewards,rl_dones,rl_actions = rl_dataset
        states = np.concatenate(rand_states,rl_states)
        next_states = np.concatenate(rand_next_states,rl_next_states) 
        rewards = np.concatenate(rand_rewards,rl_rewards)
        dones = np.concatenate(rand_dones,rl_dones)
        actions = np.concatenate(rand_actions,rl_actions)
    else:
        states, next_states, rewards, dones, actions_onehot = random_dataset

    # Separate out into training and validation sets
    X_train_states, X_val_states = states[:int(-num_examples_added*1/5)], states[int(-num_examples_added*1/5):]
    X_train_act, X_val_act = actions_onehot[:int(-num_examples_added*1/5)], actions_onehot[int(-num_examples_added*1/5):]
    
    y_env_train, y_env_val = next_states[:int(-num_examples_added*1/5)], next_states[int(-num_examples_added*1/5):]
    y_env_train, y_env_val = y_env_train - X_train_states, y_env_val - X_val_states
    
    y_rew_train, y_rew_val = dones[:int(-num_examples_added*1/5)], dones[int(-num_examples_added*1/5):]
    
    # Shuffles all the training data in the same way
    X_train_states, X_train_act, y_env_train, y_rew_train = unison_shuffled_copies(X_train_states, X_train_act, y_env_train,y_rew_train)   

    env_train_data, env_val_data = (X_train_states, X_train_act, y_env_train), (X_val_states, X_val_act, y_env_val)
    rew_train_data, rew_val_data = (X_train_states, X_train_act, y_rew_train), (X_val_states, X_val_act, y_rew_val)

    return env_train_data, env_val_data, rew_train_data, rew_val_data

'''
random_dataset = gather_random_trajectories(1000,'MsPacmanNoFrameskip-v4',encoder)
num_examples = len(states_list)
rl_dataset = []
env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(random_dataset,rl_dataset,num_examples)
'''

"\nrandom_dataset = gather_random_trajectories(1000,'MsPacmanNoFrameskip-v4',encoder)\nnum_examples = len(states_list)\nrl_dataset = []\nenv_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(random_dataset,rl_dataset,num_examples)\n"

In [0]:
def normalise(train_data, val_data, scaler = None): # Nawid - Used to normalise each dimension individually
    if scaler is None:
        scaler = StandardScaler()
        train_data = scaler.fit_transform(train_data)
    else: 
        train_data = scaler.transform(train_data)
        
    val_data = scaler.transform(val_data)
    return train_data, val_data, scaler

def normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data,X_env_obs_scaler = None,y_env_scaler=None,X_rew_obs_scaler= None):
    # Unpack data
    (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    (X_rew_train_obs, X_rew_train_act, y_rew_train), (X_rew_val_obs, X_rew_val_act, y_rew_val) = rew_train_data, rew_val_data
    
    # Normalise training and validation data
    X_env_train_obs,X_env_val_obs, X_env_obs_scaler =  normalise(X_env_train_obs, X_env_val_obs, X_env_obs_scaler)
    y_env_train, y_env_val, y_env_scaler = normalise(y_env_train, y_env_val, y_env_scaler)
    X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler = normalise(X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler)
    
    # Concatentates the normalised states with the one hot vector for the actions
    X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
    X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)
    X_rew_train = np.concatenate((X_rew_train_obs,X_rew_train_act),axis=1)
    X_rew_val = np.concatenate((X_rew_val_obs,X_rew_val_act),axis=1)

    # Pack data tuples
    env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
    rew_train_data, rew_val_data = (X_rew_train, y_rew_train),(X_rew_val, y_rew_val)

    return env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_obs_scaler, y_env_scaler, X_rew_obs_scaler

#normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)

# Train model

In [0]:
def train_model(train_data,val_data,model,batch_size, max_model_iter, optimizer, device,early_stopper,desired_model='Env_model'):
    ''' 
    General function to train either of the two models
    '''
    # Unpack data
    (X_train, y_train), (X_val, y_val) =  train_data, val_data
    losses_env = []

    # Choose loss function based on what type of model is training
    if desired_model =='Env_model': # Nawid -  Decides which loss function to use
        loss_function = model_MSEloss
    else:
        loss_function = model_BCEloss

    # go through max_model iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0,len(X_train), batch_size): # Nawid- Batch size is the step size
            if len(X_train) > mb + BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                y_mb = y_train[mb:mb+BATCH_SIZE]
                #X_mb += np.random.normal(loc = 0, scale = 0.001, size= X_mb.shape)

                optimizer.zero_grad()
                # forward pass of model to compute the output
                pred_mb = model(torch.tensor(X_mb).to(device))
                
                # compute the loss
                loss = loss_function(pred_mb,y_mb,device) #Nawid-  Uses Mse loss if dynamics model or uses BCE loss if reward/done model
                wandb.log({'{} Training loss'.format(desired_model):loss.cpu().detach().numpy()})
                # backward pass
                loss.backward()
                # optimization step
                optimizer.step()
        
        # Nawid - Calculate the validation loss after each epoch
        model.eval()
        with torch.no_grad():
            pred_val = model(torch.tensor(X_val).to(device))
            val_loss = loss_function(pred_val, y_val,device)
            wandb.log({'{} Validation loss'.format(desired_model):val_loss})
        

        # Checks whether to early stop after each epoch
        early_stopper(val_loss,model)
        if early_stopper.early_stop:
            print("Early stopping")
            break

# Controller

In [0]:
def multi_model_based_control_oracle_state(rew_model, real_obs, num_sequences, horizon_length, sample_action, norm,device):
    '''
    Use a random-sampling shooting method generating random action sequences. The first action with the highest reward of the entire sequence is returned
    '''
    oracle_env = gym.make('MsPacman')
    best_reward = -1e9
    best_next_action = []

    (env_input_scaler, env_output_scaler, rew_input_scaler) =norm
    m_obs = np.array([real_obs for _ in range(num_sequences)])

    # array that contains the rewards for all the sequence
    unroll_rewards = np.zeros((num_sequences,1))
    first_sampled_actions = []

    rew_model.eval()

    # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
    ## i.e roll a given number of trajectories in a single batch (to increase speed)
    
    for t in range(horizon_length):
        # sample actions for each sequence
        sampled_actions = np.array([np.random.randint(1, n_actions) for _ in range(num_sequences)]) # Nawid - Choosing actions on a smaller action space
        #sampled_actions = np.array([sample_action() for _ in range(num_sequences)])
        sampled_actions_one_hot = np.array([one_hot(action) for action in sampled_actions])

        if isinstance(env_model,NNDynamicsModel): # Nawid-  If the env model is a neural net
            #print('using dynamics model')
            m_obs_env_scaled = env_input_scaler.transform(m_obs)
            env_model_input = np.concatenate([m_obs_env_scaled, sampled_actions_one_hot], axis = 1)
            # compute the next state for each sequence
            pred_obs = env_model(torch.tensor(env_model_input).to(device))

            # inverse scaler transformation
            pred_obs = env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
            # add previous observation
            next_obs = pred_obs + m_obs
        else:






In [0]:
def multi_model_based_control(env_model, rew_model, real_obs, num_sequences, horizon_length, sample_action, norm,device):
    '''
    Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
    '''
    best_reward = -1e9
    best_next_action = []
    
    (env_input_scaler, env_output_scaler, rew_input_scaler) =norm
    m_obs = np.array([real_obs for _ in range(num_sequences)])
    
    # array that contains the rewards for all the sequence
    unroll_rewards = np.zeros((num_sequences, 1)) 
    first_sampled_actions = []

    env_model.eval()
    rew_model.eval()

    # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
    ## i.e roll a given number of trajectories in a single batch (to increase speed)
    
    for t in range(horizon_length):
        # sample actions for each sequence
        sampled_actions = np.array([sample_action() for _ in range(num_sequences)])
        sampled_actions_one_hot = np.array([one_hot(action) for action in sampled_actions])


        if isinstance(env_model,NNDynamicsModel): # Nawid-  If the env model is a neural net
            #print('using dynamics model')
            m_obs_env_scaled = env_input_scaler.transform(m_obs)
            env_model_input = np.concatenate([m_obs_env_scaled, sampled_actions_one_hot], axis = 1)
            # compute the next state for each sequence
            pred_obs = env_model(torch.tensor(env_model_input).to(device))

            # inverse scaler transformation
            pred_obs = env_output_scaler.inverse_transform(pred_obs.cpu().detach().numpy())
            # add previous observation
            next_obs = pred_obs + m_obs
        else:
            #print('using oracle state')            
            next_obs = env_model.predict_states(m_obs,sampled_actions) # Nawid - Able to obtain the next state directly rather than predicting a change in states

        if isinstance(rew_model, NNRewardModel):
            m_obs_rew_scaled = rew_input_scaler.transform(m_obs)
            rew_model_input = np.concatenate([m_obs_rew_scaled, sampled_actions_one_hot], axis = 1)
            pred_rew = rew_model(torch.tensor(rew_model_input).to(device)) # Nawid -  I believe I do not need to rescale for a True of false situation
            unroll_rewards +=  (1 - pred_rew.cpu().detach().numpy())
        else:
            #print('using oracle reward')
            pred_rew = rew_model.predict_reward(m_obs, sampled_actions,next_obs)
            unroll_rewards += pred_rew

        m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards
    
        if t ==0:
            first_sampled_actions = sampled_actions
        
    env_model.train()
    rew_model.train()

    # Best the position of the sequence with the higher reward
    arg_best_reward = np.argmax(unroll_rewards)
    best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
    # take the first action of this sequence
    best_action = first_sampled_actions[arg_best_reward]
    #best_action =  np.squeeze(best_action)
    return best_action, best_sum_reward

# Saving config

In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.train_model_iter = TRAIN_ITER_MODEL 
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR
config.env_lr = ENV_LEARNING_RATE
config.rew_lr = REW_LEARNING_RATE
config.no_actions = n_actions

# Main

Dynamics model training

In [0]:
# gather the dataset of random sequences

encoder = NatureCNN(observation_channels,all_defaults)
encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_55.pt'))
encoder.to(device)
encoder.eval()
rand_dataset = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
rl_dataset = []
num_examples_added = len(rand_dataset)
env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset,rl_dataset,num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
norm = (X_env_state_scaler, y_env_scaler, X_rew_state_scaler)

env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector
env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(env_model, log="all")
env_model_name = 'Env_model'+ '_' + date_time
early_stopping_env = EarlyStopping(patience=5, verbose=True, wandb=wandb, name=env_model_name)

for n_iter in range(AGGR_ITER):
    if early_stopping_env.early_stop:
        print('Early stopping')
        break
    train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

  4%|▎         | 2/55 [00:00<00:18,  2.91it/s]

Validation accuracy increased for Env_model_27_13.53.44  (0.000000 --> 1.118334).  Saving model ...
EarlyStopping for Env_model_27_13.53.44 counter: 1 out of 5


  7%|▋         | 4/55 [00:00<00:11,  4.37it/s]

EarlyStopping for Env_model_27_13.53.44 counter: 2 out of 5
EarlyStopping for Env_model_27_13.53.44 counter: 3 out of 5


  9%|▉         | 5/55 [00:01<00:10,  4.69it/s]

EarlyStopping for Env_model_27_13.53.44 counter: 4 out of 5
EarlyStopping for Env_model_27_13.53.44 counter: 5 out of 5
Env_model_27_13.53.44 has stopped
Early stopping
Early stopping


Reward model training

In [0]:
# Instantiate reward model
env = gym.make('MsPacmanNoFrameskip-v4')
encoder = ImpalaCNN(observation_channels,all_defaults)
encoder.load_state_dict(torch.load('/content/MsPacmanNoFrameskip-v4_95.pt'))
encoder.to(device)
encoder.eval()

rew_model = NNRewardModel(n_actions + feature_size,reward_dim).to(device)
rew_optimizer = torch.optim.Adam(rew_model.parameters(), REW_LEARNING_RATE)
wandb.watch(rew_model, log="all")
desired_model = 'Reward_model'
model_name = desired_model +'_'+ date_time
early_stopping_rew = EarlyStopping(patience=7, verbose=True, wandb=wandb, name=model_name)

game_reward = 0
rand_dataset = gather_random_trajectories(10000,'MsPacmanNoFrameskip-v4',encoder)
print('Does it work past here')
rl_dataset = []
num_examples_added = len(rand_dataset)

env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset, rl_dataset, num_examples_added)
norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
print('Is data normalised?')
#env_train_data, env_val_data, rew_train_data, rew_val_data = collate_data(rand_dataset,rl_dataset,num_examples_added)

for n_iter in range(AGGR_ITER):
    if early_stopping_reward.early_stop:
        print('Early stopping')
        break
    #norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler)
    print('Does the model train')
    train_model(norm_rew_train_data, norm_rew_val_data, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_rew)

  0%|          | 0/55 [00:00<?, ?it/s]

rewards list (10000, 1)
dones list (10000, 1)
Does it work past here
Is data normalised?
Does the model train


  5%|▌         | 3/55 [00:00<00:29,  1.78it/s]

Validation accuracy increased for Reward_model_2_9.26.46  (0.000000 --> 0.230905).  Saving model ...
EarlyStopping for Reward_model_2_9.26.46 counter: 1 out of 7
EarlyStopping for Reward_model_2_9.26.46 counter: 2 out of 7


  9%|▉         | 5/55 [00:01<00:21,  2.37it/s]

...

EarlyStopping for Reward_model_2_9.26.46 counter: 11 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 12 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train


  0%|          | 0/55 [00:00<?, ?it/s]

EarlyStopping for Reward_model_2_9.26.46 counter: 13 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 14 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train


  0%|          | 0/55 [00:00<?, ?it/s]

EarlyStopping for Reward_model_2_9.26.46 counter: 15 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping
Does the model train
EarlyStopping for Reward_model_2_9.26.46 counter: 16 out of 7
Reward_model_2_9.26.46 has stopped
Early stopping


# OLD CODE

In [0]:
def gather_random_trajectories(num_traj,ENV_NAME,mode = 'ATARIARI', encoder = None):
    '''
    states_list = []
    next_states_list = []
    rewards_list = []
    dones_list = []
    actions_list = []
    '''
    dataset_random = []
    #Env name could either be the RAM case or the generic case
    env = AtariARIWrapper(gym.make(ENV_NAME)) 

    for n in range(num_traj):
        obs = env.reset()
        repeated_initial = True # Nawid- Used to represent the initial state
        initial_info_labels = env.labels()
        info_labels = env.labels() # Nawid -  Used to get the current state
        while True:
            sampled_action = np.random.randint(0,n_actions)
            sampled_action_one_hot = one_hot(sampled_action)
            next_obs, reward, done, next_info = env.step(sampled_action)
            next_info_labels = next_info['labels']

            if repeated_initial: # If the initial state is repeating
                if initial_info_labels == next_info_labels: # Current state is still the same as the initial state
                    pass
                else:
                    repeated_initial = False
                    
                    state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                    reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                    dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                    
                    obs = next_obs
                    info_labels = next_info_labels

                    # Go into a new state and from the previous state and the current state as this is a transition between the states

            else:
                # Save data as this is when the initial state and the next state are not identical
                state, next_state = state_collection(obs, next_obs,info_labels, next_info_labels)
                reward, done = reward_collection(reward, done, info_labels, next_info_labels)
                dataset_random.append([state, next_state,reward,done,sampled_action_one_hot])
                
                obs = next_obs
                info_labels = next_info_labels

                if not initial_info_labels['num_lives'] == info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
                    print('change in lives')
                    repeated_initial = True



In [0]:
def state_collection(obs, next_obs, info_labels, next_info_labels,mode='ATARIARI', encoder = None):
    if mode == 'ATARIARI':
        state = []
        next_state = []
        i = 0 
        for key in info_labels :
            if i < 14: # Nawid - Only the first 14 info are crucial I believe
                state.append(info_labels[key])
                next_state.append(info_labels[key])
                i +=1
            else:
                state = np.array(state)
                next_state = np.array(next_state)
                return state, next_state
    
    elif mode == 'STDIM': # Encoding is the state
        assert encoder is not None
        with torch.no_grad():
            encoder.eval()
            states = encoder(obs.float().to(device) / 255)
            next_states = encoder(next_obs.float().to(device) / 255)
            return states, next_states

    else: # Image observations are the state or the RAM labels
        return obs, next_obs

'''
env = AtariARIWrapper(gym.make('MsPacmanNoFrameskip-v4')) 
info_labels = env.labels()
#print(info_labels)
state = []
print(info_labels)
for key in info_labels:
    state.append(info_labels[key])

state = np.array(state)
compact_state = state[0:14]
print(compact_state)
'''